<a href="https://colab.research.google.com/github/Flymshady/traffic_model/blob/main/traffic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import os




In [2]:
tf.__version__

'2.3.0'

In [3]:
# white backgrounds plots
import plotly.io as pio
pio.templates.default = 'plotly_white'

In [4]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [5]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
fid = drive.ListFile({'q':"title='trafficset.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('trafficset.zip')

In [9]:
import zipfile

with zipfile.ZipFile("trafficset.zip", 'r') as zip_ref:
  zip_ref.extractall()

# Nová sekce